## FDTD

In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("..")

import nidn
import torch

## Single layer TiO2

Configuring the simulation

In [ ]:
cfg = nidn.load_default_cfg()
# Set the number of frequencies to simulate for
cfg.N_freq = 12
#Number of layers with materials
cfg.N_layers = 1
# Define the thickness of each layer
cfg.PER_LAYER_THICKNESS=[1.0]
#Smallest wavelength
cfg.physical_wavelength_range[0]=3e-7
#Largest wavelength
cfg.physical_wavelength_range[1]=1e-6
#Convert wavelengths to normalized frequencies used by the layer builder
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution,
)
# Init layer builder
layer_builder = nidn.LayerBuilder(cfg)
#init epsiln values
eps_grid = torch.zeros(1, 1, cfg.N_layers, cfg.N_freq, dtype=torch.cfloat)
# Set first layer to titanium oxide
eps_grid[:, :, 0, :] = layer_builder.build_uniform_layer("titanium_oxide")

# Choose FDTD method, TRCWA other option
cfg.solver = "FDTD"
# Set number of time steps in FDTD
cfg.FDTD_niter = 400
# Set number of decimal in printing
torch.set_printoptions(8)
# Choose pulse type (continuous, hanning or ricker)
cfg.FDTD_pulse_type = 'continuous'
nidn.set_log_level('DEBUG')
# Choose source type (line or point)
cfg.FDTD_source_type = 'line'


In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
#Plot the spectrum
nidn.plot_spectrum(cfg,R,T)

## Multiple layers

In [ ]:
cfg = nidn.load_default_cfg()
# Set the number of frequencies to simulate for
cfg.N_freq = 12
#Number of layers with materials
cfg.N_layers = 3
# Define the thickness of each layer, in default units
cfg.PER_LAYER_THICKNESS=[1.0, 1.0, 1.0]
#Smallest wavelength
cfg.physical_wavelength_range[0]=3e-7
#Largest wavelength
cfg.physical_wavelength_range[1]=15e-7
#init epsiln values
eps_grid = torch.zeros(1, 1, cfg.N_layers, cfg.N_freq, dtype=torch.cfloat)
#Convert wavelengths to normalized frequencies used by the layer builder
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution,
)
# Choose FDTD method, TRCWA other option
cfg.solver = "FDTD"
# Init layer builder
layer_builder = nidn.LayerBuilder(cfg)
# Set the three layers to the desired material
eps_grid[:, :, 0, :] = layer_builder.build_uniform_layer("titanium_oxide")
eps_grid[:, :, 1, :] = layer_builder.build_uniform_layer("gallium_arsenide")
eps_grid[:, :, 2, :] = layer_builder.build_uniform_layer("silicon_nitride")
# Set number of time steps in FDTD
cfg.FDTD_niter = 800
# Set number of decimal in printing
torch.set_printoptions(8)
# Choose pulse type
cfg.FDTD_pulse_type = 'continuous'

In [ ]:
# Plot the abs values of the produced grid
nidn.plot_epsilon_grid(eps_grid,cfg)

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
#Plot the spectrum
nidn.plot_spectrum(cfg,R,T)